## DER Stress Analysis
Validate our DER stress analysis formulas by comparing stresses against a traditional tetrahedral FEM simulation.
We compare in pure bending, pure twisting, pure stretching, and twisting + stretching
(Unfortunately, combinations of bending and twisting/stretching are tricky to impose in the volumetric case.)

In [ ]:
import sys
sys.path.append('..')
import elastic_rods
import numpy as np
from typing import NamedTuple
from bending_validation import suppress_stdout as so
from tri_mesh_viewer import TriMeshViewer

In [ ]:
rodWidth = 520
npts = 199
midpt = (npts + 1) // 2
thetaOffset = 3 * npts

In [ ]:
# Note: the ellipse cross-section used to construct the tetrahedral mesh below has only 20 subdivisons
# (determined by the "visualization resolution" of `CrossSections::Ellipse`).
# This introduces significant discretization error, making the energy *lower* and
# the stress *higher* than the true values!

In [ ]:
pts = np.pad(np.linspace(-rodWidth / 2, rodWidth / 2, npts)[:,np.newaxis], [(0, 0), (0, 2)], mode='constant')
r = elastic_rods.ElasticRod(pts)
mat = elastic_rods.RodMaterial('rectangle', 2000, 0.3, [12, 8], stiffAxis=elastic_rods.StiffAxis.D2, keepCrossSectionMesh=True)
r.setMaterial(mat)

In [ ]:
import meshing, mesh

In [ ]:
V, _ = mat.crossSection().boundary(False)
V.append(V[0])
R = np.array([[0, -1], [1, 0]])
V = [np.array(V) @ R.T]

V, T = meshing.tetrahedralize_extruded_polylines(V, [], 520, 10)
R = np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]])
m = mesh.Mesh(V @ R.T, T, degree=2)

In [ ]:
import elastic_solid, energy, loads
import field_sampler

In [ ]:
es = elastic_solid.ElasticSolid(m, energy.CorotatedIsotropicLinearElastic(3, 2000, 0.3))

In [ ]:
v_es = TriMeshViewer(es)
v_es.show()

In [ ]:
v_es.showWireframe(False)

In [ ]:
import scipy.spatial

In [ ]:
twistAngle = np.pi / 4
x_strain = 0.01
test = 'bend'
test = 'twist'
test = 'stretch'
test = 'stretchtwist'

In [ ]:
import scipy, sparse_matrices
def getAverageEndcapPosMap(xval):
    N = m.nodes()
    beVols = m.boundaryElementVolumes()
    weights = np.zeros((3 * m.numNodes(), 3))
    totalWeight = 0
    intBdryPhi = m.integratedBoundaryShapeFunctions()
    for bei, be in enumerate(m.boundaryElementNodes()):
        if np.any(np.abs(N[be, 0] - xval) > 1e-10): continue
        weights[3 * be + 0, 0] += beVols[bei] * intBdryPhi
        weights[3 * be + 1, 1] += beVols[bei] * intBdryPhi
        weights[3 * be + 2, 2] += beVols[bei] * intBdryPhi
        totalWeight += beVols[bei]
    if totalWeight != 0: weights /= totalWeight

    dsm_scipy = scipy.sparse.csc_matrix(weights.transpose())
    dsm = sparse_matrices.SuiteSparseMatrix()
    dsm.m, dsm.n = dsm_scipy.shape
    dsm.Ap = dsm_scipy.indptr
    dsm.Ai = dsm_scipy.indices
    dsm.Ax = dsm_scipy.data
    dsm.nz = len(dsm_scipy.data)
    return dsm

In [ ]:
import py_newton_optimizer
opts = py_newton_optimizer.NewtonOptimizerOptions()
opts.gradTol = 1e-7

k = 160000
l = []
if (test == 'bend'):
    dsm = es.deformationSamplerMatrix([[-260, 0, 0], [0, 0, 0], [260, 0, 0]])
    s = loads.Springs(es, dsm, [0,  10, 0,
                                0, -10, 0,
                                0,  10, 0], stiffnesses=k * np.array([0, 1, 0, 1, 1, 1, 0, 1, 1]))
    l = [s]
if ('twist' in test):
    leftEndCapNodes = m.nodes()[(np.abs(m.nodes()[:, 0] - -260) < 1e-8)]
    rightEndCapNodes = m.nodes()[(np.abs(m.nodes()[:, 0] - 260) < 1e-8)]
    twistedRightEndcapNodes = rightEndCapNodes @ scipy.spatial.transform.Rotation.from_rotvec([twistAngle, 0, 0]).as_matrix().T
    restPts = np.vstack(([[-260, 0, 0]], leftEndCapNodes, rightEndCapNodes))
    defoPts = np.vstack(([[-260, 0, 0]], leftEndCapNodes, twistedRightEndcapNodes))
    stiffnesses = k * np.ones_like(defoPts)
    stiffnesses[1:, 0] = 0 # Remove all "x" springs but the first (needed to remove rigid motion)
    if ('stretch' in test): stiffnesses[0, 0] = 0 # Stretching applies different pins to the x coordinates, so remove that spring too...
    dsm = es.deformationSamplerMatrix(restPts)
    s = loads.Springs(es, dsm, defoPts.ravel(), stiffnesses.ravel())
    l = [s]
if ('stretch' in test):
    s1 = loads.Springs(es, getAverageEndcapPosMap(-260), [-260 * (1 + x_strain), 0, 0], [k, k, k])
    s2 = loads.Springs(es, getAverageEndcapPosMap( 260), [ 260 * (1 + x_strain), 0, 0], [k, k, k])
    l += [s1, s2]
es.computeEquilibrium(l, opts=opts)
v_es.update()

In [ ]:
ST = elastic_rods.CrossSectionStressAnalysis.StressType
st = ST.MaxPrincipal

In [ ]:
def deviatoricPart(sigma):
    return sigma - np.trace(sigma) / 3 * np.identity(3)
def stressMeasure(sigma): 
    lambdas = np.sort(np.linalg.eigvalsh(sigma))
    if (st == ST.MaxMag):       return lambdas[0] if abs(lambdas[0]) > abs(lambdas[2]) else lambdas[2]
    if (st == ST.MaxPrincipal): return lambdas[2]
    if (st == ST.MinPrincipal): return lambdas[0]
    if (st == ST.VonMises):     return np.linalg.norm(np.sqrt(3/2) * deviatoricPart(sigma).ravel())

stressMeasures = np.array([stressMeasure(sigma) for sigma in es.vertexCauchyStresses()])
v_es.update(scalarField=stressMeasures)

In [ ]:
rigidMotionVars  = [3 * midpt, 3 * midpt + 2] # pin x and z translation
rigidMotionVars += [2]                        # pin rotation around y axis (z comp. of arbitrary vtx)
rigidMotionVars += [thetaOffset]              # pin rotation around x axis

In [ ]:
x = r.getDoFs()
dirichletVars = []
if (test=='bend'):
    dirichletVars = [1, 3 * midpt + 1, 3 * (npts - 1) + 1, len(x) - 1]
    x[dirichletVars] = [10, -10, 10, 0.0]
if ('twist' in test):
    dirichletVars = [1, len(x) - 1]
    x[dirichletVars] = [0, twistAngle]
if ('stretch' in test):
    rigidMotionVars = rigidMotionVars[1:]
    dirichletVars += [0, 3 * (npts - 1)]
    x[0]              = (1 + x_strain) * -260
    x[3 * (npts - 1)] = (1 + x_strain) *  260
r.setDoFs(x)
fixedVars = rigidMotionVars + dirichletVars

In [ ]:
import py_newton_optimizer
opts = py_newton_optimizer.NewtonOptimizerOptions()
opts.niter = 1000
opts.useIdentityMetric = False
opts.useNegativeCurvatureDirection = True
opts.gradTol = 1e-4
opts.verbose = 0
forces = []
elastic_rods.compute_equilibrium(r, fixedVars=fixedVars, options=opts);

In [ ]:
es.energy(), r.energy()

In [ ]:
np.abs(stressMeasures).max(), r.maxStresses(st).max()

In [ ]:
p = 26
r.surfaceStressLpNorm(ST.VonMises, p), es.surfaceStressLpNorm(p)

In [ ]:
from matplotlib import pyplot as plt

# Stress Visualization

In [ ]:
from tri_mesh_viewer import TriMeshViewer
vmv = r.stressVisualization(True, True, ST.VonMises)
v = TriMeshViewer(vmv[0], scalarField=vmv[1].ravel())
v.show()